Benefits of Fine Tuning

* Transfer Learning
* Time and resource efficiency
* Tailored Response
* Task specific adaptation

Pitfalls of fine tuning

* Overfitting: Avoid using a small dataset or extending training epochs excessively
* Underfitting: ensure sufficient training and an appropriate learning rate to enable adequate learning
* catastrophic Forgetting: Prevent the model from losing its initial broad knowledge which hindered performance on various nlp task
* Data leakage: keep training and validation datasets seperate


Three main approaches of fine tuning models:

    * self supervised fine tuning: Here the model learns to predict missing words in a large unlabeled dataset such as next words or masked words.
    * supervised fine tuning: the model is fine tuned using labeled data from the target task, improving its performance on specific tasks like sentiment analysis.
    * Reinforcement learning from human feedback: the model is adjusted based on explicit feedback from human. 


    * `Hybrid fine tuning`: combining multiple techniques can further enhance model performance
    

`Direct performance optimization:` emerging popular approach that focuses on optimizing large language models directly based on `human preferences`. some of it's worth mentioning features are:

    * simplicity: extremely focuses on aligning model outputs with human preferences and judgements
    DPO requires no reward training. so no need to train an additional reward model
    DPO can achieve faster convergence due to its reliance on direct feedback
    

Supervised fine tuning approaches:

1. Full fine tuning: All parameters are tuned for the specific task

2. Parameter efficient fine tuning (PEFT) [more efficient]: fine tuning without modifying most of the originial parameters

## Pre-training LLMs with Hugging Face

#### Installing Required Libraries

In [4]:
!pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 torch==2.1.0+cu118
!pip install pmdarima -U
!pip install --upgrade pmdarima==2.0.2


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y
  Using cached pmdarima-2.0.2.tar.gz (630 kB)
  Preparing metadata (setup.py) ... one
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [13 lines of output]
      /private/var/folders/y2/33vgfdz176b018l37jlk7z3m0000gn/T/pip-install-fmj6uchg/pmdarima_dd7e3246e877456ca69c0df2f0151978/setup.py:15: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import parse_version
      Partial import of pmdarima during the build process.
      Requirements: ['joblib>=0.11', 'Cython>=0.29,!=0.29.18,!=0.29.31',

In [5]:
!pip install transformers == 4.40.0
!pip install -U git+https://github.com/huggingface/transformers
!pip install --user datasets 2.15.0
!pip install --user portalocker>=2.0.0
!pip install -q -U git+https://github.com/huggingface/accelerate.git

zsh:1: = not found
  Cloning https://github.com/huggingface/transformers to /private/var/folders/y2/33vgfdz176b018l37jlk7z3m0000gn/T/pip-req-build-0znspwb9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/y2/33vgfdz176b018l37jlk7z3m0000gn/T/pip-req-build-0znspwb9
  Resolved https://github.com/huggingface/transformers to commit d2ae766836d1862a814ccd016306727111627673
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... one
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
ERROR: Could not find a version that satisfies the requirement 2.15.0 (from versions: none)
ERROR: No matching distribution found for 2.15.0
zsh:1: 2.0.0 not found


In [6]:
!pip install -q -U accelerate
!pip install --user torch==2.3.0
!pip install -U torchvision
!pip install --user protobug ==3.20.*

  Using cached torch-2.3.0-cp312-none-macosx_11_0_arm64.whl.metadata (26 kB)
Using cached torch-2.3.0-cp312-none-macosx_11_0_arm64.whl (61.0 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.17.2 requires torch==2.2.2, but you have torch 2.3.0 which is incompatible.
torchvision 0.22.1 requires torch==2.7.1, but you have torch 2.3.0 which is incompatible.
  Using cached torch-2.7.1-cp312-none-macosx_11_0_arm64.whl.metadata (29 kB)
Using cached torch-2.7.1-cp312-none-macosx_11_0_arm64.whl (68.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [14]:
import torch
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments,Trainer,TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
#from datasets import load_dataset


from tqdm.auto import tqdm
import math
import time
import os

import warnings
def warn(*args,**kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings('ignore')


In [12]:
!pip install --user dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.5 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.17.2 requires torch==2.2.2, but you have torch 2.7.1 which is incompatible.


In [15]:
# Set the environment variable tokenizers_parallelism to 'false'
import os
os.environ["TOKENIZERS_PARALLELISM"] = 'false'

In [16]:
model = AutoModelForCausalLM.from_pretrained('facebook/opt-350m')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

pine = pipeline(task = "text-generation", model = model, tokenizer = tokenizer)
print(pine("This movie was really")[0]["generated_text"])

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Device set to use mps:0
Both `max_new_tokens` (=256) and `max_length`(=21) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


This movie was really fun. I liked it a lot. I'm not sure I would've liked it if it wasn't for the music and the story.
I like the music but I couldn't help but feel a bit underwhelmed by the story.
